## 초기 세팅 및 데이터 준비

In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import os

# OpenAI API를 활용 시
# === 필수: OPENAI_API_KEY ===
# os.environ["OPENAI_API_KEY"] = "S13P32S101-f2a0e414-d636-44a5-982a-8230a5edb0db"

# === 선택: 팀 전용 게이트웨이 ===
# 예: "https://your-team-endpoint.example.com/v1"
# os.environ["OPENAI_BASE_URL"] = "https://gms.ssafy.io/gmsapi/api.openai.com/v1"

# os.environ.setdefault("OPENAI_MODEL", "gpt-4o")

BASE_MODEL_ID = os.getenv("BASE_MODEL_ID", "Qwen/Qwen3-4B-Instruct-2507")


In [3]:

import json, csv, random, time
from typing import List, Dict, Any

import pandas as pd
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_exponential

# OpenAI SDK (신/구버전 호환)
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1")
BASE_MODEL_ID = os.getenv("BASE_MODEL_ID", "Qwen/Qwen3-4B-Instruct-2507")
random.seed(7)

print("OPENAI_API_KEY", OPENAI_API_KEY)
print("OPENAI_BASE_URL", OPENAI_BASE_URL)
print("OPENAI_MODEL:", OPENAI_MODEL)
print("BASE_MODEL_ID:", BASE_MODEL_ID)


OPENAI_API_KEY 
OPENAI_BASE_URL https://api.openai.com/v1
OPENAI_MODEL: gpt-4.1
BASE_MODEL_ID: Qwen/Qwen3-4B-Instruct-2507


In [4]:
import pandas as pd
from pathlib import Path
import re

base_dir = Path("test")

# 모든 txt 파일 경로 불러오기 (재귀적으로)
txt_files = list(base_dir.rglob("*.txt"))

sentences = []

for file in txt_files:
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:  # 빈 줄 제외
                # "A." 또는 "B." 제거 (앞에 공백이 있어도 제거)
                line = re.sub(r'^[AB]\.\s*', '', line)

                sentences.append({
                    "file_path": str(file),
                    "sentence": line.strip("A.")
                })

df = pd.DataFrame(sentences)

display(df.tail(20))
print(f"총 문장 수: {len(df)}")


,file_path,sentence
480518,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 계산서 담당자 이메일로 국세청에서 자동 전송됩니다
480519,test/TS_3.education.zip/03.비용환불문의/education3_0...,사업자번호 및 메일 주소 정확하게 기재 부탁드립니다
480520,test/TS_3.education.zip/03.비용환불문의/education3_0...,그럼 제가 개인적으로 출력할 수는 없을까요
480521,test/TS_3.education.zip/03.비용환불문의/education3_0...,로그인 후 마이페이지에서 계산서 출력 가능합니다
480522,test/TS_3.education.zip/03.비용환불문의/education3_0...,교육 전 계산서 선발행이 필요하시면 사전 발급 신청하시면 됩니다
480523,test/TS_3.education.zip/03.비용환불문의/education3_0...,참고로 카드결제인 경우 계산서 발급은 불가합니다
480524,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 자세하게 설명해 주셔서 감사합니다
480525,test/TS_3.education.zip/03.비용환불문의/education3_0...,편안한 오후 되십시오 상담사 #@이름#이었습니다
480526,test/TS_3.education.zip/03.비용환불문의/education3_0...,안녕하세요 #@소속# 상담사 #@이름#입니다
480527,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 안녕하세요 케이크 만들기 신청했는데요


총 문장 수: 480538


In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
def run_pipe(batch):
    outputs = pipe(batch["sentence"], batch_size=8, max_new_tokens=512)
    return {"result": [o[0]["generated_text"] for o in outputs]}

In [6]:
# 데이터 후처리
# 원래 라벨링 전에 해야하는데, 실수로 먼저 돌려서 토큰 아끼기 위해 후처리

# 중복된 행 제거 (완전히 같은 행 기준)
df = df.drop_duplicates()

# 만약 특정 컬럼 기준으로만 중복 제거하고 싶다면
df = df.drop_duplicates(subset=['file_path', 'sentence'])

# 인덱스 리셋 (선택사항)
df = df.reset_index(drop=True)

# .ipynb_checkpoints 가 포함된 행 제거
df = df[~df['file_path'].str.contains('/.ipynb', na=False)]

# 인덱스 리셋 (선택)
df = df.reset_index(drop=True)

display(df)

,file_path,sentence
0,test/TS_4.tourism.zip/05.관광/tourism5_2093.txt,기쁨 가득한 상담원 #@이름#입니다
1,test/TS_4.tourism.zip/05.관광/tourism5_2093.txt,네 안녕하세요 문의드릴 게 있어서요
2,test/TS_4.tourism.zip/05.관광/tourism5_2093.txt,네 어떤 내용이신가요
3,test/TS_4.tourism.zip/05.관광/tourism5_2093.txt,이번에 세계술문화박물관 리쿼리움에 방문해보려고 하는데요
4,test/TS_4.tourism.zip/05.관광/tourism5_2093.txt,박물관 관람료가 어떻게 되나요
...,...,...
480424,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 기다려 주셔서 감사합니다 #@소속#계좌 6만원으로 입금하셨네요
480425,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 맞아요 #@소속#으로 입금했어요
480426,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 고객님 입금확인 문자는 개별 발송됩니다 조금만 기다려 주세요
480427,test/TS_3.education.zip/03.비용환불문의/education3_0...,네 확인 감사합니다


In [7]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
display(dataset)

Dataset({
    features: ['file_path', 'sentence'],
    num_rows: 480429
})

## 모델 로드 및 라벨링 로직

In [7]:
from typing import Optional
import torch
from transformers import pipeline

def safe_parse_json(payload: Any) -> Dict[str, Any]:
    """
    Robust JSON extractor:
    - payload가 dict/list/str 어떤 형태여도 처리
    - assistant.content만 추출
    - 바깥 { ... } 슬라이스
    - 흔한 깨짐 복구: 개행/탭/작은따옴표/트레일링 콤마
    """
    # 1) assistant 텍스트만 뽑기
    text = ""

    # 케이스 A: {'role':'assistant','content':'{...}'}
    if isinstance(payload, dict) and "content" in payload:
        text = str(payload.get("content", ""))

    # 케이스 B: [{'generated_text': '...'}] 또는 [{'generated_text': [{'role':..., 'content': '...'}]}]
    elif isinstance(payload, list) and payload:
        first = payload[0]
        gen = first.get("generated_text") if isinstance(first, dict) else None
        if isinstance(gen, str):
            text = gen
        elif isinstance(gen, list):
            # role 리스트라면 뒤에서부터 assistant content 찾기
            for msg in reversed(gen):
                if isinstance(msg, dict) and msg.get("role") == "assistant":
                    text = msg.get("content", "")
                    break
            if not text:
                for msg in gen:
                    if isinstance(msg, dict) and "content" in msg:
                        text = msg["content"]
                        break
        else:
            # list인데 포맷이 다르면 문자열로 캐스팅
            text = str(first)

    # 케이스 C: 이미 문자열
    elif isinstance(payload, str):
        text = payload
    else:
        text = str(payload or "")

    s = text.strip()
    if not s:
        return {}

    # 2) 바깥쪽 { ... }만 슬라이스
    if "{" in s and "}" in s:
        s = s[s.find("{"): s.rfind("}") + 1]

    # 3) 1차 파싱 시도
    try:
        return json.loads(s)
    except Exception:
        pass

    # 4) 복구: 개행/탭 제거, 작은따옴표 -> 큰따옴표
    s_fix = s.replace("\n", " ").replace("\t", " ")
    # JSON 내 한국어/문장부호엔 영향 적으니 전체 치환 허용
    s_fix = s_fix.replace("'", '"')

    # 5) 복구: 트레일링 콤마 제거 (", }", ", ]")
    s_fix = re.sub(r",\s*([}\]])", r"\1", s_fix)

    # 6) 최종 파싱 시도
    try:
        return json.loads(s_fix)
    except Exception:
        return {}


### 프롬프트 및 few shot 예시

In [8]:
# ✅ 분류 기준 (객체 루트로 통일)
SYSTEM_PROMPT = """너는 '일정 관련 문장(1) / 비일정(0)' 분류기다.
반드시 아래 JSON만 출력해.
필드:
- label: 0 또는 1
- reason: 한 줄 한국어 이유
- certainty: 0~100 정수 (확신도)
- time_cue: 시간/날짜 단서 포함 여부 (true/false)
- schedule_cue: 예약/방문/배정/가능 등 일정 단서 포함 여부 (true/false)
출력 예: {"label":1,"reason":"내일 11시 방문 요청","certainty":92,"time_cue":true,"schedule_cue":true}
규칙: JSON 외 불필요한 텍스트 금지.
기준: 시간/날짜/장소/예약 생성·변경·확인 요청은 1, 단순 안내/사과/인사는 0.
애매하면 단서 없으면 0, 단서 2개↑면 1.
"""


# FEW_SHOTS = [
#     {"role":"user","content":"문장: '열한 시 삼십 분에 기사 연락 드리고 방문 드릴게요'"},
#     {"role":"assistant","content":'{"label":1,"reason":"방문 일정에 대해 공유하는 내용","certainty":95,"time_cue":true,"schedule_cue":true}'},

#     {"role":"user","content":"문장: '같은 부분이면 비용이 안 나옵니다'"},
#     {"role":"assistant","content":'{"label":0,"reason":"단순 안내 문장","certainty":90,"time_cue":false,"schedule_cue":false}'},

#     {"role":"user","content":"문장: '내일 2시에 콘티 리뷰 시간 괜찮으세요?'"},
#     {"role":"assistant","content":'{"label":1,"reason":"업무에 대한 리뷰 일정 제안","certainty":93,"time_cue":true,"schedule_cue":true}'},

#     {"role":"user","content":"문장: '네 지금 바로 갈 거예요'"},
#     {"role":"assistant","content":'{"label":0,"reason":"단순 현재 상황 언급","certainty":85,"time_cue":false,"schedule_cue":false}'},

#     {"role":"user","content":"문장: '오늘 잉킹 세션 취소합니다'"},
#     {"role":"assistant","content":'{"label":1,"reason":"기존 일정 취소 언급","certainty":95,"time_cue":true,"schedule_cue":true}'},

#     {"role":"user","content":"문장: '전화번호 어떻게 되세요'"},
#     {"role":"assistant","content":'{"label":0,"reason":"단순 연락처 문의","certainty":90,"time_cue":false,"schedule_cue":false}'},
# ]



In [11]:
# ==== OpenAI 클라이언트 (한 번만 생성) ====
# OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")  # 커스텀 게이트웨이 쓰면 설정, 아니면 None
# client = OpenAI(
#     api_key=os.getenv("OPENAI_API_KEY"),
#     base_url=OPENAI_BASE_URL if OPENAI_BASE_URL else None
# )

# ==== 오픈소스 모델 로드 (한 번만 생성) ====
pipe = None  # 전역 변수
def get_LLM():
    global pipe
    if pipe is None:
        if not BASE_MODEL_ID:
            raise RuntimeError("BASE_MODEL_ID not set.")
        from transformers import AutoTokenizer
        
        # 토크나이저 별도 로드하여 패딩 설정
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
        tokenizer.padding_side = "left"
        
        # pad_token이 없으면 eos_token을 사용
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        pipe = pipeline("text-generation", model=BASE_MODEL_ID, tokenizer=tokenizer)
    return pipe

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=10))
def begin_pipeline(system_prompt: str, user_prompts: list[str]) ->  List[List[str]]:
    pipe = get_LLM()
    if pipe:
        try:
            batch = [
                [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": p}
                ]
                for p in user_prompts
            ]
            outputs = pipe(batch,
                           max_new_tokens=2048,
                           batch_size=16,
                           temperature=0.7,
                           num_return_sequences=3,  # ← n번 생성
                           do_sample=True,          # ← 필수 (샘플링 허용)
                          )
            # print(outputs)
            texts = []
            for output in outputs:
                texts.append([out.get('generated_text')[2]["content"] for out in output])
            return texts
        except Exception as e:
            print(e)
def run_pipe(batch):
    # batch["sentence"]는 리스트
    outs = begin_pipeline(SYSTEM_PROMPT, batch["sentence"])
    return {"raw": outs}

In [12]:
resp = begin_pipeline(SYSTEM_PROMPT, ["아까 작업한 내용을 그대로 가는게 좋겠어", "감사합니다", "내일 오후 4시까지 마무리해주세요."])
print(resp)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


[['{"label":0,"reason":"작업 내용 유지 요청은 일정과 무관","certainty":85,"time_cue":false,"schedule_cue":false}', '{"label":0,"reason":"작업 내용 그대로 가는 것에 대한 안내입니다","certainty":70,"time_cue":false,"schedule_cue":false}', '{"label":0,"reason":"작업 내용 그대로 가는 것에 대한 제안이나 예약 단서 없음","certainty":70,"time_cue":false,"schedule_cue":false}'], ['{"label":0,"reason":"단순 인사 문장","certainty":100,"time_cue":false,"schedule_cue":false}', '{"label":0,"reason":"단순 인사입니다","certainty":100,"time_cue":false,"schedule_cue":false}', '{"label":0,"reason":"단순 인사 문장","certainty":100,"time_cue":false,"schedule_cue":false}'], ['{"label":1,"reason":"내일 오후 4시까지 마무리 요청","certainty":95,"time_cue":true,"schedule_cue":true}', '{"label":1,"reason":"내일 오후 4시까지 마무리 요청","certainty":95,"time_cue":true,"schedule_cue":true}', '{"label":1,"reason":"내일 오후 4시까지 마무리 요청","certainty":95,"time_cue":true,"schedule_cue":true}']]


In [13]:
# pip install openai pandas tqdm tenacity
import os, json, re, statistics
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from openai import OpenAI
from collections import Counter

def _safe_parse_json(text: str):
    # 코드블록/잡텍스트 섞여도 {} 첫/끝만 추출
    m = re.search(r'\{.*\}', text, re.S)
    if not m: 
        return None
    try:
        return json.loads(m.group(0))
    except:
        return None

def classify_n(sentence: str, model="gpt-4o-mini", n=5, temperature=0.3, LLM=True):
    # 1) n 지원이면 한 번에
    if LLM:
        resp = begin_pipeline(SYSTEM_PROMPT, sentence)
    else:
        resp = client.chat.completions.create(
            model=model,
            temperature=temperature,
            n=n,
            messages=[
                {"role":"system","content":SYSTEM_PROMPT},
                {"role":"user","content":sentence}
            ]
        )
    outs = []
    for ch in resp:
        data = _safe_parse_json(ch)
        if data and "label" in data:
            outs.append({
                "label": int(data.get("label", 0)),
                "certainty": int(data.get("certainty", 50)),
                "time_cue": bool(data.get("time_cue", False)),
                "schedule_cue": bool(data.get("schedule_cue", False)),
                "reason": data.get("reason", "")
            })
    return outs

def vote_result(outs, conservative_default=0):
    # 다수결 + 동률시 확신도 평균 높은 쪽
    if not outs:
        return {"label": conservative_default, "agree_rate": 0.0, "certainty_avg": 0, "outs": []}
    labels = [o["label"] for o in outs]
    cnt = Counter(labels)
    top_label, top_count = cnt.most_common(1)[0]
    agree_rate = top_count / len(outs)
    # 동률 처리
    if len([1 for v in cnt.values() if v == top_count]) > 1:
        # 확신도 평균으로 승부
        cert_avg = {
            lab: statistics.mean([o["certainty"] for o in outs if o["label"]==lab]) 
            for lab in cnt.keys()
        }
        # 같으면 보수적으로 0
        if cert_avg.get(1,0) == cert_avg.get(0,0):
            top_label = conservative_default
        else:
            top_label = max(cert_avg.items(), key=lambda x: x[1])[0]
    certainty_avg = int(statistics.mean([o["certainty"] for o in outs if o["label"]==top_label]))
    return {
        "label": int(top_label),
        "agree_rate": round(agree_rate, 3),
        "certainty_avg": certainty_avg,
        "outs": outs
    }

In [25]:
from tqdm import tqdm
import pandas as pd

def classify_n(sentence: str, n=5, temperature=0.7, LLM=True):
    """
    단일 문장에 대해 n번 생성하여 분류 결과 반환
    """
    if LLM:
        # begin_pipeline은 리스트를 받으므로 단일 문장을 리스트로 감싸기
        resp = begin_pipeline(SYSTEM_PROMPT, [sentence])
        # resp[0]은 해당 문장에 대한 n개 결과 리스트
        raw_outputs = resp[0] if resp and len(resp) > 0 else []
    else:
        # OpenAI API 사용하는 경우 (현재는 사용하지 않음)
        raw_outputs = []
    
    outs = []
    for text in raw_outputs:
        data = safe_parse_json(text)
        if data and "label" in data:
            outs.append({
                "label": int(data.get("label", 0)),
                "certainty": int(data.get("certainty", 50)),
                "time_cue": bool(data.get("time_cue", False)),
                "schedule_cue": bool(data.get("schedule_cue", False)),
                "reason": data.get("reason", "")
            })
    return outs

def label_with_vote_batch_optimized(sentences, n=5, batch_size=8):
    """
    datasets.Dataset을 사용하여 GPU 효율성 극대화
    """
    # Dataset 생성
    dataset = Dataset.from_dict({"sentence": sentences})
    
    def process_batch(batch):
        try:
            # begin_pipeline으로 배치 처리
            outs_batch_raw = begin_pipeline(SYSTEM_PROMPT, batch["sentence"])
            
            results = []
            for sentence_outputs in outs_batch_raw:
                outs = []
                for text in sentence_outputs:
                    data = safe_parse_json(text)
                    if data and "label" in data:
                        outs.append({
                            "label": int(data.get("label", 0)),
                            "certainty": int(data.get("certainty", 50)),
                            "time_cue": bool(data.get("time_cue", False)),
                            "schedule_cue": bool(data.get("schedule_cue", False)),
                            "reason": data.get("reason", "")
                        })
                
                voted = vote_result(outs)
                voted["accept"] = (voted["agree_rate"] >= 0.8 and voted["certainty_avg"] >= 70)
                results.append(voted)
            
            return {
                "label_pred": [r["label"] for r in results],
                "agree_rate": [r["agree_rate"] for r in results],
                "certainty_avg": [r["certainty_avg"] for r in results],
                "accept": [r["accept"] for r in results]
            }
        except Exception as e:
            print(f"Error processing batch: {e}")
            batch_size = len(batch["sentence"])
            return {
                "label_pred": [0] * batch_size,
                "agree_rate": [0.0] * batch_size,
                "certainty_avg": [0] * batch_size,
                "accept": [False] * batch_size
            }
    
    # 배치 처리 실행
    processed_dataset = dataset.map(
        process_batch,
        batched=True,
        batch_size=batch_size,
        desc="Labeling with GPU optimization"
    )
    
    return processed_dataset

def run_labeling_optimized(df: pd.DataFrame, text_col="sentence", n=5, batch_size=16):
    """
    Dataset을 사용한 최적화된 라벨링
    """
    sentences = df[text_col].astype(str).tolist()
    processed_dataset = label_with_vote_batch_optimized(sentences, n=n, batch_size=batch_size)
    
    # 결과를 DataFrame에 추가
    df["label_pred"] = processed_dataset["label_pred"]
    df["agree_rate"] = processed_dataset["agree_rate"]
    df["certainty_avg"] = processed_dataset["certainty_avg"]
    df["accept"] = processed_dataset["accept"]
    
    return df

In [22]:
# from tqdm import tqdm
# tqdm.pandas(desc="Labeling sentences")

# def label_with_vote(sentence: str, n=5):
#     outs = classify_n(sentence, n=n, temperature=0.3, LLM=True)
#     voted = vote_result(outs)
#     voted["accept"] = (voted["agree_rate"] >= 0.8 and voted["certainty_avg"] >= 70)
#     return voted
    
# def run_labeling(df: pd.DataFrame, text_col="sentence", n=5):
#     results = df[text_col].progress_apply(lambda s: label_with_vote(str(s), n=n))
#     df["label_pred"] = results.apply(lambda r: r["label"])
#     df["agree_rate"] = results.apply(lambda r: r["agree_rate"])
#     df["certainty_avg"] = results.apply(lambda r: r["certainty_avg"])
#     df["accept"] = results.apply(lambda r: r["accept"])
#     return df

In [30]:
# 전체 데이터의 10% 또는 특정 개수만 사용
sample_size = min(50000, len(df))  # 5만개 또는 전체 중 작은 값
df_sample = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

In [31]:
df_labeled = run_labeling_optimized(df_sample, batch_size=16)

# 저장 및 확인
df_labeled.to_csv("labeled_sentences_voted_every_data.csv", index=False)
df_labeled.head(10)

Labeling with GPU optimization:   0%|          | 0/50000 [00:00<?, ? examples/s]

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_4.tourism.zip/04.레저/tourism4_1596.txt,전화받았습니다 #@소속#입니다,0,1.0,83,True
1,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,그것도 해주시고 다 해주세요 왜냐면 저도 이제 몇 번을 참은 거라서요,0,1.0,75,True
2,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,네 다른 건의사항 있으십니까,0,1.0,85,True
3,test/TS_3.education.zip/03.비용환불문의/education3_0...,아 그래요 #@기타#문고나 #@기타#문고에도 없어요,0,1.0,76,True
4,test/TS_1.shopping.zip/02.제품사용문의/shopping2_221...,감사합니다 고객님 #@소속# 고객센터였습니다,0,1.0,100,True
5,test/TS_1.shopping.zip/07.온오프라인안내/shopping7_11...,다른 문의 사항 있으실까요,0,1.0,100,True
6,test/TS_1.shopping.zip/05.환불반품교환/shopping5_119...,맞습니다 문의사항 하나 남겨주세요,0,1.0,88,True
7,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,아뇨 됐습니다 수고하세요,0,1.0,98,True
8,test/TS_1.shopping.zip/04.배송/shopping4_1359.txt,이미 상품을 준비하는 단계이기에 옵션 변경이 어렵습니다,0,1.0,85,True
9,test/TS_1.shopping.zip/06.이벤트/shopping6_0747.txt,수고하십니다 문의할 게 있어서요,0,1.0,80,True


In [128]:
# # df는 이미 sentence 컬럼을 포함한 상태라고 가정
# df_labeled = label_with_llm(df, text_col="sentence", batch_size=20, model=OPENAI_MODEL)

# 저장 및 확인
# df_labeled.to_csv("labeled_sentences_voted.csv", index=False)
# df_labeled.head(10)

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 반갑습니다 상담사 #@이름#입니다,0,1.0,95,True
1,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,네 아까 제가 지금 상담을 하는 건데 들으세요,0,1.0,84,True
2,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,#@소속#에서 판매를 어떤 물건을 하든지 하게 되면은 판매 정책과 사업부가 있죠,0,1.0,87,True
3,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,상담하는 거는 그 사업을 위해서 상담하는 거고요,0,1.0,85,True
4,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그니까 저희는 유지관리를 상담해 드리는 상담실입니다,0,1.0,85,True
5,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,사업을 진행하는 데에서는 고객님과 통화하시는 부분이 아니고요,0,1.0,85,True
6,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,저희가 내용을 전달해 드리는 겁니다,0,1.0,85,True
7,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그렇죠 상담하시는 그러니까 그 상담을 그거 밖에 안 되잖아요,0,1.0,85,True
8,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,그래서 그 이외에 총괄적으로 사업을 지휘하는 데가 어딥니까,0,1.0,85,True
9,test/TS_1.shopping/01.AS문의/shopping1_0625.txt,거기의 전화번호 알려달라니까,0,1.0,85,True


In [32]:
# df_labeled = pd.read_csv("labeled_sentences.csv")
display(df_labeled)

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_4.tourism.zip/04.레저/tourism4_1596.txt,전화받았습니다 #@소속#입니다,0,1.0,83,True
1,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,그것도 해주시고 다 해주세요 왜냐면 저도 이제 몇 번을 참은 거라서요,0,1.0,75,True
2,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,네 다른 건의사항 있으십니까,0,1.0,85,True
3,test/TS_3.education.zip/03.비용환불문의/education3_0...,아 그래요 #@기타#문고나 #@기타#문고에도 없어요,0,1.0,76,True
4,test/TS_1.shopping.zip/02.제품사용문의/shopping2_221...,감사합니다 고객님 #@소속# 고객센터였습니다,0,1.0,100,True
...,...,...,...,...,...,...
49995,test/TS_4.tourism.zip/04.레저/tourism4_1036.txt,빗길 조심하시고 행복한 하루 되세요 #@이름#이었습니다,0,1.0,100,True
49996,test/TS_2.civil_complaint.zip/02.절차문의/civil co...,그 쪽은 자기네 관할 도로가 아니고 국도래요 국도,0,1.0,83,True
49997,test/TS_1.shopping.zip/01.AS문의/shopping1_2716.txt,예 우선 센터에서 하게되면요,0,1.0,70,True
49998,test/TS_4.tourism.zip/01.숙박/tourism1_0796.txt,열린펜션민박이요 한번 찾아봐야겠네요,0,1.0,75,True


In [33]:
# 데이터 후처리
# 원래 라벨링 전에 해야하는데, 실수로 먼저 돌려서 토큰 아끼기 위해 후처리

# 중복된 행 제거 (완전히 같은 행 기준)
df_labeled = df_labeled.drop_duplicates()

# 만약 특정 컬럼 기준으로만 중복 제거하고 싶다면
df_labeled = df_labeled.drop_duplicates(subset=['file_path', 'sentence'])

# 인덱스 리셋 (선택사항)
df_labeled = df_labeled.reset_index(drop=True)

# .ipynb_checkpoints 가 포함된 행 제거
df_labeled = df_labeled[~df_labeled['file_path'].str.contains('/.ipynb', na=False)]

# 인덱스 리셋 (선택)
df_labeled = df_labeled.reset_index(drop=True)

In [34]:
# 일정 관련 대화 수, 비율 점검
schedule_labeled = df_labeled["label_pred"] == 1

display(df_labeled)
display(df_labeled[schedule_labeled])
print("일정으로 labeling 된 데이터 수 :", len(df_labeled[schedule_labeled]["file_path"]))

,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
0,test/TS_4.tourism.zip/04.레저/tourism4_1596.txt,전화받았습니다 #@소속#입니다,0,1.0,83,True
1,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,그것도 해주시고 다 해주세요 왜냐면 저도 이제 몇 번을 참은 거라서요,0,1.0,75,True
2,test/TS_2.civil_complaint.zip/04.민원신고/civil co...,네 다른 건의사항 있으십니까,0,1.0,85,True
3,test/TS_3.education.zip/03.비용환불문의/education3_0...,아 그래요 #@기타#문고나 #@기타#문고에도 없어요,0,1.0,76,True
4,test/TS_1.shopping.zip/02.제품사용문의/shopping2_221...,감사합니다 고객님 #@소속# 고객센터였습니다,0,1.0,100,True
...,...,...,...,...,...,...
49995,test/TS_4.tourism.zip/04.레저/tourism4_1036.txt,빗길 조심하시고 행복한 하루 되세요 #@이름#이었습니다,0,1.0,100,True
49996,test/TS_2.civil_complaint.zip/02.절차문의/civil co...,그 쪽은 자기네 관할 도로가 아니고 국도래요 국도,0,1.0,83,True
49997,test/TS_1.shopping.zip/01.AS문의/shopping1_2716.txt,예 우선 센터에서 하게되면요,0,1.0,70,True
49998,test/TS_4.tourism.zip/01.숙박/tourism1_0796.txt,열린펜션민박이요 한번 찾아봐야겠네요,0,1.0,75,True


,file_path,sentence,label_pred,agree_rate,certainty_avg,accept
17,test/TS_1.shopping.zip/01.AS문의/shopping1_0957.txt,오늘 일찍 좀 오셨음 좋겠는데요,1,1.0,85,True
51,test/TS_4.tourism.zip/01.숙박/tourism1_1350.txt,객실 2개 이상을 예약해도 안될까요,1,1.0,85,True
78,test/TS_1.shopping.zip/01.AS문의/shopping1_2857.txt,네 그 제품은 모니터라서요 기사님 방문 원하시나요,1,1.0,85,True
81,test/TS_1.shopping.zip/04.배송/shopping4_1032.txt,확인 감사합니다 상품 확인해보니까 금일 출고예정이라고 하세요,1,1.0,85,True
101,test/TS_1.shopping.zip/04.배송/shopping4_0730.txt,그 마스크 벌써 신청해 놨는데 안 오네,1,1.0,85,True
...,...,...,...,...,...,...
49897,test/TS_4.tourism.zip/05.관광/tourism5_2300.txt,네 감사합니다 저 그리고 주말에 가려고 하는데요,1,1.0,75,True
49920,test/TS_4.tourism.zip/05.관광/tourism5_1070.txt,네 고객님 대금굴 예매하셨습니까,1,1.0,85,True
49934,test/TS_1.shopping.zip/01.AS문의/shopping1_1159.txt,오전과 오후 모두 일정 가능하신데 언제가 편하십니까,1,1.0,95,True
49942,test/TS_2.civil_complaint.zip/01.부서안내/civil co...,혹시 나중에 참석 여부를 좀 여쭤볼려고요,1,1.0,85,True


일정으로 labeling 된 데이터 수 : 2916
